# ATARI Deep-Q Learning

## Introdução

Código apresenta uma implementação da rede neural **Deep Q-Learning**,
desenvolvida pelo grupo de pesquisadores da Deep Mind e, publicado no seguinte [artigo](https://arxiv.org/pdf/1312.5602.pdf) em 2013.


## Deep-Q Learning

À medida que um agente realiza ações e se move através de um ambiente, ele aprende a mapear o estado observado do ambiente para uma ação.
Um agente escolherá uma ação em um determinado estado com base em um "valor Q", que é uma recompensa ponderada com base na maior recompensa de longo prazo esperada.
Um Agente Q-Learning aprende a realizar sua tarefa de forma que a ação recomendada maximize as possíveis recompensas futuras.
Este método é considerado um método "Off-Policy", ou seja, seus valores Q são atualizados assumindo que a melhor ação foi escolhida, mesmo que a melhor ação não tenha sido escolhida.


## Atari Breakout

No jogo em questão, o jogador poode movimentar uma barra para o lado esquerdo, ou direito da tela, procurando rebater uma bola que então destroi blocos distribuídos em camadas na parte superior da tela. O objetivo do jogo é remover todos os blocos da tela. Neste ambiente, o agente deve aprender a controlar o tabuleiro movendo-se para a esquerda e para a direita, devolvendo a bola e removendo todos os blocos sem que a bola passe no tabuleiro.

## Inputs

Semelhante a um jogador humano, o agente recebe uma série de imagens representando o estado em tempo real do jogo, e deve a partir destas imagens inferir se o mesmo deve se mover para esquerda ou direita da tela. Em outras palavras, o agente recebe uma série de imagens da tela e, deve determinar para qual dos dois lados possíveis ele deve se mover.


:::{attention}

Em uma máquina moderna, é necessário de cerca de 24 horas de treino, até que o agente atinga um nível razoável de experiência no jogo.

:::


## Instruções


Nenhum tipo de instrução é passada ao agente no início do jogo. Apenas os possíveis movimentos legais que o agente pode escolher são informadas.
Durante o treino, melhoras no score e tempo até o GameOver são pontuadas, de modo a incentivar o agente, na tomada de escolhas que façam com que o mesmo
se aproxime do objetivo final.


## Resultado


O video a seguir mostra o agente no início do treino. O vdeo em questão foi capurado mais ou menos uma hora após inicio do treino.

![](docs/_static/DEEPQ_TRAIN_START.gif)


Já no próximo vídeo, o agente possui por volta de 30 horas de experiência com o jogo.


![](docs/_static/DEEPQ_TRAIN_MID_END.gif)




# Objetivo

O objetivo deste notebook é apresentar o potêncial dos modelos de análise de imagem apresentados no presente tutorial, para análise de videos também. Embora o exemplo em questão não apresente um grando valor em termos de utilide no mundo real, ele apresenta um caso de uso extremamente utilizado por outras áreas, como para a aálise de "self-driving" cars.


:::{warning}

Configurar o jogo da Atari requer níveis avançado de conheciment em ciencia de dados, mas também de gerenciamento de pacotes externos.

:::


In [1]:
from baselines.common.atari_wrappers import make_atari, wrap_deepmind
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Configuration paramaters for the whole setup
seed = 42

# Discount factor for past rewards
gamma = 0.99

# Epsilon greedy parameter
epsilon = 1.0
epsilon_min = 0.1  # Minimum epsilon greedy parameter.
epsilon_max = 1.0  # Maximum epsilon greedy parameter.

# Rate at which to reduce chance of random action being taken.
epsilon_interval = epsilon_max - epsilon_min

# Size of batch taken from replay buffer
batch_size = 32 
max_steps_per_episode = 10_000


In [17]:
env = gym.make('BreakoutNoFrameskip-v4', render_mode='human')

/Users/erikingwersen/Library/Caches/pypoetry/virtualenvs/pyimg-oSL_kg38-py3.8/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: obs_type "image" should be replaced with the image type, one of: rgb, grayscale
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [19]:
# Use the Baseline Atari environment because of Deepmind helper functions
# env = make_atari("BreakoutNoFrameskip-v4")
env = gym.make('BreakoutNoFrameskip-v4', render_mode='human')
# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)
env.seed(seed)

/Users/erikingwersen/Library/Caches/pypoetry/virtualenvs/pyimg-oSL_kg38-py3.8/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(3444837047, 2669555309)

In [20]:
num_actions = 4

def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(84, 84, 4,))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.MaxPooling2D(2, 2)(layer1)
    layer3 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer2)
    # layer4 = layers.MaxPooling2D(2, 2)(layer3)
    layer5 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer3)

    layer6 = layers.Flatten()(layer5)

    layer7 = layers.Dense(512, activation="relu")(layer6)
    action = layers.Dense(num_actions, activation="linear")(layer7)

    return keras.Model(inputs=inputs, outputs=action)


In [21]:
# The first model makes the predictions for Q-values which are used to
# make a action.
model = create_q_model()

# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

In [ ]:
# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# Number of frames to take random action and observe output
epsilon_random_frames = 50000

# Number of frames for exploration
epsilon_greedy_frames = 1000000.0

# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000

# Train the model after 4 actions
update_after_actions = 4

# How often to update the target network
update_target_network = 10_000

# Using huber loss for stability
loss_function = keras.losses.Huber()


while True:  # Run until solved
    
    state = np.array(env.reset())
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # gym.make(timestep, render_mode='human')
        
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample)
            
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]

    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 40:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 0.29 at episode 70, frame count 10000
running reward: 0.17 at episode 149, frame count 20000
running reward: 0.17 at episode 229, frame count 30000
running reward: 0.24 at episode 297, frame count 40000
running reward: 0.23 at episode 367, frame count 50000
running reward: 0.18 at episode 448, frame count 60000
running reward: 0.19 at episode 527, frame count 70000
running reward: 0.16 at episode 607, frame count 80000
running reward: 0.28 at episode 678, frame count 90000
running reward: 0.14 at episode 764, frame count 100000
running reward: 0.13 at episode 846, frame count 110000
running reward: 0.27 at episode 918, frame count 120000
running reward: 0.17 at episode 1000, frame count 130000
running reward: 0.15 at episode 1081, frame count 140000
running reward: 0.17 at episode 1159, frame count 150000
running reward: 0.25 at episode 1230, frame count 160000
running reward: 0.20 at episode 1304, frame count 170000
running reward: 0.24 at episode 1376, frame count 180